In [81]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pickle

## Backend

In [97]:
# backend
# loading single coin models
def coin_model_instance():
    window_size = 60
    model_coin = Sequential([
        LSTM(128, return_sequences=True, input_shape=(window_size, 1)),
        Dropout(0.2),
        LSTM(128, return_sequences=False),
        Dense(units=1)
    ])
    return model_coin


def reload_model(coin):
    window_size = 60
    model_paths = [
        './checkpoints/btc_checkpoint.weights.h5',
        './checkpoints/eth_checkpoint.weights.h5',
        './checkpoints/ltc_checkpoint.weights.h5'
    ]

    model_multicoin = Sequential(
        # [LSTM(64, return_sequences=True, input_shape=(window_size, 5)),
        # Dropout(0.2),
        # LSTM(64, return_sequences=False),
        # Dense(units=1)]
        [LSTM(50, return_sequences=True, input_shape=(window_size, 5)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(units=5)]
    )
    model_multicoin.load_weights('./checkpoints/merge_checkpointv2.weights.h5')

    coin_id = -1
    if coin == 'BTC':
        coin_id = 0
    elif coin == "LTC":
        coin_id = 1
    elif coin == 'ETH':
        coin_id = 2

    coin_model = coin_model_instance()
    coin_model.load_weights(model_paths[coin_id])
    # print(model_multicoin.summary())
    # print(coin_model.summary())
    return model_multicoin, coin_model


def reload_scaler(coin):
    coin_paths = [
        './checkpoints/btc_scaler.pkl',
        './checkpoints/ltc_scaler.pkl',
        './checkpoints/eth_scaler.pkl',
    ]

    # multi-coin
    with open('./checkpoints/merge_scaler.pkl', 'rb') as f:
        merge_scaler = pickle.load(f)
        f.close()
    
    # single coin
    coin_id = -1
    if coin == 'BTC':
        coin_id = 0
    elif coin == "LTC":
        coin_id = 1
    elif coin == 'ETH':
        coin_id = 2
    
    with open(coin_paths[coin_id], 'rb') as f_coin:
        coin_scaler = pickle.load(f_coin)
        f_coin.close()

    return merge_scaler, coin_scaler

def fetch_current_data(coin='BTC'):
    btc = yf.Ticker('BTC-USD')
    ltc = yf.Ticker('LTC-USD')
    eth = yf.Ticker('ETH-USD')

    btc_hist = btc.history(period='60D')
    ltc_hist = ltc.history(period='60D')
    eth_hist = eth.history(period='60D')

    coin_list = [btc_hist, ltc_hist, eth_hist]
    for c in coin_list:
        if 'Stock Splits' in c.columns:
            c.drop(columns=['Stock Splits'], inplace=True)
        if 'Dividends' in c.columns:
            c.drop(columns=['Dividends'], inplace=True)

    '''
    For single coin, require only close price data.
    For multi-coin, require all 5 features
    '''

    btc_data = btc_hist
    eth_data = eth_hist
    ltc_data = ltc_hist

    if coin == "BTC":
        return btc_data
    elif coin == "ETH":
        return eth_data
    elif coin == 'LTC':
        return ltc_data


In [98]:
def forecast(forecast_days=[1, 5, 10, 30], coin='BTC', target_col=3):
    # Fetch initial inputs
    X_merge_init = fetch_current_data(coin)  # shape (60, 5)
    current_price_value = X_merge_init['Close'].iloc[-1]
    X_coin_init = X_merge_init['Close']      # assuming shape (60, 5) too

    # Reload scalers and models
    merg_scaler, coin_scaler = reload_scaler(coin)
    merge_model, coin_model = reload_model(coin)

    # Constants
    total_features = X_merge_init.shape[-1]

    # Convert to NumPy arrays
    X_input_multicoin = X_merge_init.to_numpy()
    X_input_coin = X_merge_init['Close'].to_numpy()

    # Ensure shapes are correct
    X_input_multicoin = X_input_multicoin.reshape(1, 60, total_features)
    X_input_coin = X_input_coin.reshape(1, 60, 1)

    # Store predictions
    forecast_result_multicoin = {}
    forecast_result_single = {}
    predicted_closes_multicoin = []
    predicted_closes_single = []

    for _ in range(1, max(forecast_days) + 1):
        # Predict scaled close values
        pred_scaled_multicoin = merge_model.predict(X_input_multicoin, verbose=0)
        pred_scaled_coin = coin_model.predict(X_input_coin, verbose=0)

        # print(pred_scaled_multicoin)
        # print(X_input_coin.shape)

        # Inverse transform
        pred_full_multicoin = merg_scaler.inverse_transform(pred_scaled_multicoin)
        pred_full_coin = coin_scaler.inverse_transform(pred_scaled_coin)

        # print(pred_full_multicoin)

        pred_close_multicoin = pred_full_multicoin[:, target_col][0]
        pred_close_coin = pred_full_coin[0]

        predicted_closes_multicoin.append(pred_close_multicoin)
        predicted_closes_single.append(pred_close_coin)

        # Recursive input update
        next_input_multi = X_input_multicoin.copy()
        new_step_multi = pred_scaled_multicoin.flatten().copy()
        X_input_multicoin = np.concatenate([next_input_multi, new_step_multi.reshape(1, 1, total_features)], axis=1)

        next_input_coin = X_input_coin.copy()
        new_step_coin = pred_scaled_coin.flatten().copy()
        X_input_coin = np.concatenate([next_input_coin, new_step_coin.reshape(1, 1, 1)], axis=1)


    # Extract forecasts
    for d in forecast_days:
        forecast_result_multicoin[f"{d}d"] = predicted_closes_multicoin[d - 1]
        forecast_result_single[f"{d}d"] = predicted_closes_single[d - 1]

    # Return predictions and the latest observed timestep
    return forecast_result_multicoin, forecast_result_single, current_price_value



In [99]:
res = forecast()
print(type(res))
print(len(res))
type(res[0]), type(res[1]), type(res[2])

<class 'tuple'>
3


(dict, dict, numpy.float64)

In [100]:
res[0]

{'1d': np.float32(33416.223),
 '5d': np.float32(54415.453),
 '10d': np.float32(97180.164),
 '30d': np.float32(116897.33)}

In [ ]:
def generate_recommendation(predictions, current_price, current_hold=0, sell_threshold_per=250):
    """
    Generate investment recommendation based on forecasted prices, 
    current price, and user holdings, with an upper holding threshold.
    """
    sell_threshold = current_price * sell_threshold_per
    returns = {k: ((v - current_price) / current_price) * 100 for k, v in predictions.items()}

    all_negative = all(r < 0 for r in returns.values())
    immediate_gain = returns["1d"] > 1
    rebound_after_drop = returns["1d"] < -1 and returns["5d"] > 3
    long_term_gain = returns["30d"] > 5
    low_volatility = max(returns.values()) < 2

    # If user is holding too much, recommend selling on any kind of gain
    if current_hold >= sell_threshold:
        if all_negative:
            return "Sell now to minimize loss"
        elif immediate_gain or long_term_gain:
            best_day = max(returns, key=returns.get)
            return f"Sell part or all on {best_day} to book profit"
        elif low_volatility:
            return "Market is flat — consider reducing your position"
    
    # Case 1: All returns are negative
    if all_negative:
        if current_hold > 0:
            return "Sell your holdings to avoid further loss"
        else:
            return "Avoid buying now — market looks negative"

    # Case 2: Strong rebound pattern detected
    if rebound_after_drop:
        if current_hold > 0:
            return "Hold your position, expect a rebound by day 5"
        else:
            return "Buy now to benefit from rebound, sell on day 5"

    # Case 3: Immediate gain and strong 30-day return
    if immediate_gain and long_term_gain:
        if current_hold > 0:
            return "Hold and consider increasing your position"
        else:
            return "Buy now and hold for long-term gain"

    # Case 4: Short-term spike followed by dip
    if returns["1d"] > 0 and returns["5d"] < 0:
        if current_hold > 0:
            return "Sell on day 1 to take quick profit"
        else:
            return "Buy now and sell on day 1 for quick gain"

    # Case 5: Minor short-term gain but strong long-term growth
    if returns["1d"] < 1 and long_term_gain:
        if current_hold > 0:
            return "Hold and consider buying more"
        else:
            return "Buy now and hold for long-term return"

    # Case 6: All returns low
    if low_volatility:
        if current_hold > 0:
            return "Hold — not enough momentum to sell or buy more"
        else:
            return "Wait — market is not promising right now"

    # Default: Recommend best day to sell for profit
    best_day = max(returns, key=returns.get)
    if current_hold > 0:
        return f"Hold and sell on {best_day} for best return"
    else:
        return f"Buy now and sell on {best_day} for profit"


In [102]:
# def generate_recommendation(predictions, current_price, current_hold=0):
#     """
#     Generate investment recommendation based on forecasted prices, 
#     current price, and user holdings.
#     """
#     returns = {k: ((v - current_price) / current_price) * 100 for k, v in predictions.items()}
    
#     all_negative = all(r < 0 for r in returns.values())
#     immediate_gain = returns["1d"] > 1
#     rebound_after_drop = returns["1d"] < -1 and returns["5d"] > 3
#     long_term_gain = returns["30d"] > 5

#     # Case 1: All returns are negative
#     if all_negative:
#         if current_hold > 0:
#             return "Sell your holdings to avoid further loss"
#         else:
#             return "Avoid buying now — market looks negative"

#     # Case 2: Strong rebound pattern detected
#     if rebound_after_drop:
#         if current_hold > 0:
#             return "Hold your position, expect a rebound by day 5"
#         else:
#             return "Buy now to benefit from rebound, sell on day 5"

#     # Case 3: Immediate gain and strong 30-day return
#     if immediate_gain and returns["30d"] > 8:
#         if current_hold > 0:
#             return "Hold and consider increasing your position"
#         else:
#             return "Buy now and hold for long-term gain"

#     # Case 4: Short-term spike followed by dip
#     if returns["1d"] > 0 and returns["5d"] < 0:
#         if current_hold > 0:
#             return "Sell on day 1 to take quick profit"
#         else:
#             return "Buy now and sell on day 1 for quick gain"

#     # Case 5: Minor short-term gain but strong long-term growth
#     if returns["1d"] < 1 and long_term_gain:
#         if current_hold > 0:
#             return "Hold and consider buying more"
#         else:
#             return "Buy now and hold for long-term return"

#     # Case 6: All returns low
#     if max(returns.values()) < 2:
#         if current_hold > 0:
#             return "Hold — not enough momentum to sell or buy more"
#         else:
#             return "Wait — market is not promising right now"

#     # Default: Recommend best day to sell for profit
#     best_day = max(returns, key=returns.get)
#     if current_hold > 0:
#         return f"Hold and sell on {best_day} for best return"
#     else:
#         return f"Buy now and sell on {best_day} for profit"


def get_optimal_prediction(coin, user_holding):
    
    # getting forcasts
    forecast_multicoin, forecast_single, current_values = forecast(coin=coin)
    # print(forecast_multicoin, forecast_single, current_values)
    # print(current_values)
    current_price = current_values
    rec_multicoin = generate_recommendation(forecast_multicoin, current_price, user_holding)
    rec_single = generate_recommendation(forecast_single, current_price, user_holding)

    text = f"""\n
    Forcast & Recommendations for {coin}:
    Based on the Multicoin model: 
    Forcast:
                    1 Day           5 Day           10 Day          30 Day
    Close price     {float(forecast_multicoin['1d']):.4f}     {float(forecast_multicoin['5d']):.4f}     {float(forecast_multicoin['10d']):.4f}    {float(forecast_multicoin['30d']):.4f}

    Recommendation: {rec_multicoin}

    Based on the {coin} model: 
    Forcast:
                    1 Day           5 Day           10 Day          30 Day
    Close price     {float(forecast_single['1d']):.4f}     {float(forecast_single['5d']):.4f}     {float(forecast_single['10d']):.4f}    {float(forecast_single['30d']):.4f}

    Recommendation: {rec_single}
    """

    print(text)
    print("Your current holding: ", user_holding)
    print("Current coin price: ", current_values)


## Frontend

In [104]:
# frontend
print("Welcome to Crypto Guide")
print("""
In this guide, users can select and forsee the forcast of 3 different coins by inbuilt models. 
Coin selection can be made from the following: Bitcoin, Ethereum, Litecoin.
The program also gives out recommendations based on forecasts of the individual coin.

Disclaimer on Prediction Discrepancies:
Predictions made by the single-coin model and the multi-coin model may differ significantly due to differences in the data used and the assumptions each model captures.
Single coin model has been trained exclusively on the histrocial data of a single cryptocurrency. Henceforth, it is more tailored towards capturing that coin's trends, seasonality and behavior.
Mutlicoin has been trained on a merged dataset consisting historical price data of BTC, ETH and LTC. It learns the general market trend.
""")
while True:
    # try:
    user_coin = str(input('Enter coin [BTC: Bitcoin, ETH: Ethereum, LTC: Litcoin]'))
    user_holding = int(input("Enter current holding: "))
    user_coin = user_coin.strip()
    # user_coin = "BTC"
    if user_coin not in ["BTC", "ETH", "LTC"]:
        print("Coin not found!")
    else:
        get_optimal_prediction(user_coin, user_holding)

        res = int(input('Do you want to any other coin? (Yes: 1, No: 0): '))
        # res = res.strip()
        # print("res: ", res)
        if res == 1:
            continue
        else:
            break
    # except Exception as e:
    #     raise e

print("Thank you")

Welcome to Crypto Guide

In this guide, users can select and forsee the forcast of 3 different coins by inbuilt models. 
Coin selection can be made from the following: Bitcoin, Ethereum, Litecoin.
The program also gives out recommendations based on forecasts of the individual coin.

Disclaimer on Prediction Discrepancies:
Predictions made by the single-coin model and the multi-coin model may differ significantly due to differences in the data used and the assumptions each model captures.
Single coin model has been trained exclusively on the histrocial data of a single cryptocurrency. Henceforth, it is more tailored towards capturing that coin's trends, seasonality and behavior.
Mutlicoin has been trained on a merged dataset consisting historical price data of BTC, ETH and LTC. It learns the general market trend.



    Forcast & Recommendations for BTC:
    Based on the Multicoin model: 
    Forcast:
                    1 Day           5 Day           10 Day          30 Day
    Close p